In [2]:
import uuid
from IPython.display import display_javascript, display_html, display
import json

class RenderJSON(object):
    def __init__(self, json_data):
        if isinstance(json_data, dict):
            self.json_str = json.dumps(json_data)
        else:
            self.json_str = json_data
        self.uuid = str(uuid.uuid4())

    def _ipython_display_(self):
        display_html('<div id="{}" style="height: 600px; width:100%;"></div>'.format(self.uuid), raw=True)
        display_javascript("""
        require(["https://rawgit.com/caldwell/renderjson/master/renderjson.js"], function() {
        document.getElementById('%s').appendChild(renderjson(%s))
        });
        """ % (self.uuid, self.json_str), raw=True)

In [3]:
yelp_client_id = "lCMOSNkXwkYe-LuMw4k3sQ"
yelp_api = "9Y59m3VeDAEDy6JJCK3yi-JoiVNoqLRFaJlhhE1IqWNDnuv1a8T63v_mNJWOqRDR2gUL_YsGNB8Wv-XK7P1ProB1JK2MHsPi2y3ffjUG1s1r-hnKKe3jpad9CwERY3Yx"

fs_id = "UYFNAMOM3KORDEV4HZJAHATPGELIATQJFOUMZAJFDOVVXPJ3"
fs_s = "KQQ5C552HGDYAFB3MOFOM1V5JFCIMOY5ERJ445IJ0EJ234KJ"



In [4]:
import requests
import os
import pandas as pd
import numpy as np
from IPython.display import JSON
import json
from pandas import json_normalize

In [ ]:
#YELP

In [5]:
term = 'coffee'
location = 'Vancouver,BC'
radius = 1000
limit = 50


base_url = 'https://api.yelp.com/v3/businesses/search'

headers = {
         'Authorization': f'Bearer {yelp_api}'
}

params = {
    'term' : term,
    'location' : location,
    'radius': radius,
    'limit' : limit
}

res = requests.get(url=base_url, headers=headers,params=params)
result = res.json()
RenderJSON(result)


In [182]:
print(res.text)

Header overflow


In [9]:
result['businesses'][0]['coordinates']

{'latitude': 49.2632037459, 'longitude': -123.110469593}

In [6]:
yelpvan = []
for i in result['businesses']:
    busdict = {}
    busdict['name'] = i['name']
    busdict['review_count'] = i['review_count']
    busdict['rating'] = i['rating']
    busdict['address'] = i['location']['address1']
    busdict['city'] = i['location']['city']
    busdict['zip_code'] = i['location']['zip_code']
    busdict['latitude'] = i['coordinates']['latitude']
    busdict['longitude'] = i['coordinates']['longitude']
    yelpvan.append(busdict)

In [7]:
df_yvancoffee = pd.DataFrame(yelpvan)
df_yvancoffee.to_csv('yelpvancoffee.csv')

In [17]:
df_yvancoffee.head(5)

,name,review_count,rating,address,city,zip_code,latitude,longitude
0,Aperture Coffee Bar,119,4.0,243 W Broadway,Vancouver,V5Y 3L1,49.263204,-123.110470
1,Paragon Tea Room,59,4.5,2207 Cambie Street,Vancouver,V5Z 2W6,49.265648,-123.115106
2,Caffe Cittadella,150,4.0,2310 Ash Street,Vancouver,V5Z 3C2,49.264809,-123.117267
3,Modus Coffee Roasters,27,4.5,112 W Broadway,Vancouver,V5Y 1P1,49.262946,-123.107436
4,The Golden Horn Turkish Bakery & Cafe,6,5.0,2857 Cambie Street,Vancouver,V5Z 3Y8,49.259811,-123.115283


In [ ]:
#SQL

In [8]:
import sqlite3 as sqlite
from sqlalchemy import create_engine
import sqlalchemy as db

In [43]:
con = create_engine('sqlite:///coffee.db')


In [ ]:
df_yvancoffee.to_sql('y_van_coffee',con)

In [9]:
#top ten from yelp according to rating
#yelp dfs
#   df_yvancoffee
df_yv = df_yvancoffee

In [189]:
df_yv.sort_values(by=['rating'],ascending=False).head(10)


,name,review_count,rating,address,city,zip_code,latitude,longitude
49,Uno Gelato,12,5.0,601 Stamps Landing,Vancouver,V5Z 3Z1,49.269096,-123.118501
33,Cheyenne Coffee,1,5.0,,Vancouver,,49.261226,-123.113927
4,The Golden Horn Turkish Bakery & Cafe,6,5.0,2857 Cambie Street,Vancouver,V5Z 3Y8,49.259811,-123.115283
10,On the Grind - Pedalling Epic Coffee,4,5.0,,Vancouver,,49.261226,-123.113927
20,Bean Around The World Coffees,4,5.0,820 West 10th Avenue,Vancouver,V5Z 1M9,49.262265,-123.123355
12,Johnnycantsurf Coffee,1,5.0,525 W 10th Avenue,Vancouver,V5Z 1K9,49.262380,-123.116024
13,Convivial Cafe and Bakery,8,5.0,680A Leg In Boot Square,Vancouver,V5Z 4B4,49.267543,-123.119664
15,Deja Brew,51,4.5,943 W Broadway,Vancouver,V5Z 4E1,49.263660,-123.125130
39,Lemonade Gluten Free Bakery,126,4.5,3385 Cambie Street,Vancouver,V5Z 2W6,49.255157,-123.115342
30,JJ Bean Coffee Roasters,38,4.5,188 W 1st Avenue,Vancouver,V5Y 1A4,49.270150,-123.109227


In [10]:
df_yv.sort_values(by=['review_count'],ascending=False).head(10)

,name,review_count,rating,address,city,zip_code,latitude,longitude
36,Yolks,329,3.5,546 W Broadway,Vancouver,V5Z 1E9,49.263114,-123.116353
5,Marulilu Cafe,280,4.0,451 W Broadway,Vancouver,V5Y 1R4,49.263380,-123.114150
42,Whole Foods Market,243,3.5,510 W 8th Avenue,Vancouver,V5Z 1C5,49.264021,-123.115276
31,Solly's Bagelry,237,2.5,368 W 7th Avenue,Vancouver,V5Y 1M4,49.264567,-123.112492
6,Elysian Coffee,203,3.5,590 W Broadway,Vancouver,V5Z 1E9,49.262940,-123.117280
29,Passion8 Dessert Cafe,199,4.0,3010 Cambie Street,Vancouver,V5Z 2V9,49.258435,-123.114998
2,Caffe Cittadella,150,4.0,2310 Ash Street,Vancouver,V5Z 3C2,49.264809,-123.117267
39,Lemonade Gluten Free Bakery,126,4.5,3385 Cambie Street,Vancouver,V5Z 2W6,49.255157,-123.115342
0,Aperture Coffee Bar,119,4.0,243 W Broadway,Vancouver,V5Y 3L1,49.263204,-123.110470
41,Dutch Wooden Shoe Cafe,99,3.5,3292 Cambie Street,Vancouver,V5Z 2W4,49.256002,-123.114899


In [12]:
dfyv=df_yv['name'].count()
dfyv

50